In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Flatten, Dense, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom, RandomTranslation, Dropout, Rescaling
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.utils import to_categorical
from tensorflow.data import AUTOTUNE
from tensorflow.keras import models, layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Activation

import matplotlib.pyplot as plt

import os
from pathlib import Path

In [73]:
RANDOM_SEED=4
BASE_PATH = os.path.join(Path(os.getcwd()).resolve().parents[1], "Data")
assert os.path.isdir(BASE_PATH), f'Data Directory is required: {BASE_PATH}'
BATCH_SIZE = 128
EPOCHS = 2
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

tf.random.set_seed(RANDOM_SEED)

In [28]:
train_data = image_dataset_from_directory(TRAIN_PATH, image_size=(50,50), label_mode='binary',
                                  seed=RANDOM_SEED, shuffle=True, batch_size=BATCH_SIZE)
val_data = image_dataset_from_directory(VAL_PATH, image_size=(50,50), label_mode='binary',
                                  seed=RANDOM_SEED, shuffle=True, batch_size=BATCH_SIZE)
test_data = image_dataset_from_directory(TEST_PATH, image_size=(50,50), label_mode='binary',
                                  seed=RANDOM_SEED, shuffle=True, batch_size=BATCH_SIZE)

Found 176794 files belonging to 2 classes.
Found 49334 files belonging to 2 classes.
Found 51396 files belonging to 2 classes.


In [29]:
data_augmentation = Sequential([
        RandomFlip("horizontal_and_vertical", input_shape=(50, 50, 3)),
        RandomRotation(0.2),
        RandomZoom(0.3),
        RandomTranslation(height_factor=0.2, width_factor=0.2),
        Rescaling(1./255),
])


/Users/mananpatel/anaconda3/envs/pytorch/lib/python3.11/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [67]:
# Create an EfficientNetB0 model
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(50, 50, 3))

# Freeze the first half of the layers
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Define additional layers
additional_layers = Sequential([
    GlobalAveragePooling2D(),
    Dense(1, activation='sigmoid')
])

# Combine base model with additional layers
model = Sequential([
    base_model,
    additional_layers
])

# Print model summary
model.summary()


Model: "sequential_66"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ ?                      │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_65 (Sequential)      │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 2,560 (10.00 KB)

 Non-trainable params: 4,047,011 (15.44 MB)

In [68]:
# Get the last layer of the model
last_layer = model.layers[-2]

# Get the output shape of the last layer
last_layer_output_shape = last_layer.output_shape

print("Last layer output shape:", last_layer_output_shape)

Last layer output shape: (None, 2, 2, 1280)


In [78]:
whole_system = Sequential([
    data_augmentation,
    model
])

whole_system.summary()

Model: "sequential_68"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_26 (Sequential)      │ (None, 50, 50, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_66 (Sequential)      │ (None, 1)              │     4,050,852 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 3,841 (15.00 KB)

 Non-trainable params: 4,047,011 (15.44 MB)

In [79]:
checkpoint_dir = 'EffecientB0_Models'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_filepath = os.path.join(checkpoint_dir, 'checkpoint.model2.keras')

# Define the ModelCheckpoint callback
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)

# Define the EarlyStopping callback
limited_computation_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3)

In [80]:
whole_system.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy(), 
                tf.keras.metrics.F1Score(), 
                tf.keras.metrics.Recall(), 
                tf.keras.metrics.Precision()])

In [81]:
whole_system.fit(train_data, epochs=EPOCHS, validation_data=val_data, 
                batch_size=BATCH_SIZE, callbacks=[model_checkpoint_callback, 
                                                limited_computation_callback],
                # class_weight={
                # 0: 1/0.7, # weight for the majority class (inversely proportional to its frequency)
                # 1: 1/0.3}  # weight for the minority class (inversely proportional to its frequency)
                )

Epoch 1/2
5525/5525 ━━━━━━━━━━━━━━━━━━━━ 689s 123ms/step - binary_accuracy: 0.7096 - f1_score: 0.4465 - loss: 0.6057 - precision_7: 0.2893 - recall_7: 0.0071 - val_binary_accuracy: 0.6769 - val_f1_score: 0.4884 - val_loss: 0.6286 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00
Epoch 2/2
 380/5525 ━━━━━━━━━━━━━━━━━━━━ 11:42 137ms/step - binary_accuracy: 0.7088 - f1_score: 0.4489 - loss: 0.6050 - precision_7: 0.2533 - recall_7: 0.0053

KeyboardInterrupt: 

In [ ]:
whole_system.evaluate(test_data)

1258/1607 ━━━━━━━━━━━━━━━━━━━━ 28s 82ms/step - binary_accuracy: 0.2319 - f1_score: 0.3764 - loss: 0.6994 - precision_3: 0.2319 - recall_3: 1.0000

KeyboardInterrupt: 